<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/HonGong_ML_DL/11_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

### **키워드:** 앙상블 학습, 랜덤 포레스트, 엑스트라 트리, 그래디언트 부스팅

앙상블 학습이 무엇인지 알아보고 다양한 앙상블 학습 알고리즘 실습을 진행해보자.

In [4]:
# Packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

## 정형 데이터와 비정형 데이터

지금까지 우리는 CSV 파일과 같이 잘 정돈된 데이터를 다루었다. 이처럼 어떠한 구조를 가지고 있는 데이터를 **정형 데이터(Structured data)**라고 한다. 앞에서 다룬 CSV나 데이터프레임, 혹은 데이터베이스 등 개발자가 다루는 대부분의 데이터는 정형 데이터이다.

반대로 **비정형 데이터(Unstructured data)**도 있는데, 사진, 음성, 텍스트 등 사전에 정해진 형태가 없는 데이터를 지칭한다.

(반드시 그런것은 아니지만) 보통 정형 데이터에 대해 좋은 성능을 보이는 머신러닝과 달리 딥러닝의 경우 비정형 데이터에 대해서도 수준급의 성능을 보이는 것으로 알려져있고, 이는 딥러닝이 현시대의 연구 및 산업 현장 대부분에 자리 잡을 수 있는 계기를 부여했다고 해도 과언이 아니다.

머신러닝 중 일반적으로 대부분의 정형 데이터에서 수준급의 성능을 내는 알고리즘이 있는데, 이것이 바로 **앙상블 학습(Ensemble Learning)**이다.

## 랜덤 포레스트

**랜덤 포레스트(Random Forest)**는 앙상블 학습의 대표 주자 중 하나로, 안정적인 성능을 내는 덕에 널리 사용되는 알고리즘 중 하나이다.

이름에서 알 수 있듯이, 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 앙상블 시킨 모델이다. 먼저 각 트리를 학습시키기 위해 데이터를 랜덤하게 만드는데, 이때 **부트스트랩(Bootstrap)** 방식이 사용된다. 이는 '복원추출'이라고 설명할 수 있는데, 전체 데이터에서 샘플을 하나 추출 후 다시 해당 샘플을 복원시켜 다시 추출을 진행한다. 따라서, 중복되는 샘플이 추출될 수도 있다. 기본적으로, 이렇게 만들어진 부트스트랩 샘플은 훈련 세트와 크기가 같아질 때 까지 추출을 진행한다.

또한 각 노드를 분할할 때 전체 특성 중 일부 특성을 무작위로 골라 이 중 최선의 분할을 찾는다. 보통 $\sqrt{전체\ 특성}$만큼 선택을 진행한다. 선택되는 특성은 노드마다 랜덤하게 바뀌게 되며, 모든 노드에 대해 분할이 진행된다.

이후 전체 트리의 결과값을 평균하는 방식으로 최종 결과를 도출해낸다.

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 과대적합을 방지할 수 있으며, 매개변수를 따로 조정해주지 않아도 준수한 성능을 기대할 수 있다.



### 랜덤 포레스트로 와인 분류하기

In [6]:
wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

# data split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [7]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


약간 과대적합되었음을 확인할 수 있다. 보통 이 경우 하이퍼 파라미터 튜닝을 진행하지만, 여기서는 모델의 구조를 아는 것이 주목적이기도 하고 워낙 문제가 간단하여 HPO를 수행해도 크게 나아지지 않으므로 생략하겠다.

In [8]:
rf.fit(X_train, y_train)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


이전 노트북에서 진행했던 결과와는 약간 다르다. 이는 랜덤 포레스트가 특성을 랜덤하게 선택하는 성질때문인데, 이로 인해 모델은 과대적합되지 않고 일반화 성능을 높일 수 있다.

`RandomForestClassifier`에는 재밌는 기능이 하나 더 있다. 바로 자체적으로 모델을 평가할 수 있다는 점인데, 이는 부트스트랩 샘플과 관련이 있다.

중복을 허용하며 전체 데이터셋의 크기와 같은 크기의 샘플을 뽑는 부트스트랩 샘플링 방식에서는 한 번도 선별되지 않은 샘플이 남아있을 수 있는데, 이것을 **OOB샘플(Out of Bag Sample)**이라고 한다. 바로 이것을 검증 세트처럼 사용하여 평가를 수행하는 것이다.

이 점수는 `RandomForestClassifier` 클래스의 `oob_score` 매개변수를 True로 지정함으로써 얻을 수 있다.

In [9]:
rf = RandomForestClassifier(oob_score=True,  n_jobs=-1, random_state=42)

rf.fit(X_train, y_train)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

**엑스트라 트리(Extra tree)**는 랜덤 포레스트와 매우 비슷하게 동작하는 앙상블 학습 알고리즘이다. 이 또한 특성을 랜덤하게 선택하여 노드 분할에 이용하며, 랜덤 포레스트가 제공하는 매개변수 대부분을 제공한다.

하지만 랜덤 포레스트와 달리 엑스트라 트리는 부트스트랩 샘플을 이용하지 않고 원본 훈련 데이터를 사용한다. 또한 가장 좋은 분할이 아닌 랜덤 분할을 실행한다. 이렇게 하면 성능은 낮아질 수 있지만 보다 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 세트의 점수를 올릴 수 있다는 장점이 있다.

`DecisionTreeClassifier`에서 `splitter` 매개변수값을 random으로 줘도 비슷한 결과를 낼 수 있다.

In [10]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(et, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


여기서는 문제의 복잡도가 높지 않아 별 차이가 없지만, 랜덤하게 노드를 분할하는 엑스트라 트리 방식은 보통 랜덤 포레스트 방식보다 계산 속도가 빠르다.

In [11]:
et.fit(X_train, y_train)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그래디언트 부스팅

**그래디언트 부스팅(Gradient Boosting)**은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다. 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강건(robust)하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

그래디언트라는 이름처럼, 이 모델은 경사하강법을 이용하여 트리를 앙상블에 추가하는 방식으로 동작한다. 결정 트리를 조금씩 추가하면서 최적의 값에 도달할 때까지 이를 반복한다. 그래서 깊이가 얕은 결정 트리를 이용하는 것이다.

In [16]:
gb = GradientBoostingClassifier(random_state=42) # Gradient Boosting 방식은 직렬적으로 트리를 추가하며 계산하기 때문에 n_jobs 매개변수가 없다.

scores = cross_validate(gb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


앞의 두 모델과 달리 과대적합이 거의 일어나지 않았음을 확인할 수 있다.

`learning_rate`와 `n_estimator`를 조정하여 성능을 향상시킬 수 있다.

In [17]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)

scores = cross_validate(gb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [18]:
gb.fit(X_train, y_train)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


Gradient Boosting 방식에는 `subsample`이라는 매개변수가 있는데, 이것의 기본값은 1.0으로, 전체 훈련 세트를 이용한다는 의미이다. 만약 이를 낮춘다면 훈련 세트의 일부분을 사용하게 되고, 이를 이용해 미니배치 경사하강법이나 확률적 경사하강법과 같은 효과를 내도록 설계할 수 있다.

### 히스토그램 기반 그래디언트 부스팅

그래디언트 부스팅의 속도와 성능을 더욱 개선한 것으로, 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘이다.

히스토그램 기반 그래디언트 부스팅은 먼저 입력 특성을 256개의 구간으로 분할하여 최적의 분할을 매우 빠르게 찾을 수 있도록 돕는다.

In [20]:
hgb = HistGradientBoostingClassifier(random_state=42) # 부스팅 반복 횟수를 조절하려면 n_estimator 대신 max_iter를 사용하면 된다

scores = cross_validate(hgb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [25]:
hgb.score(X_test, y_test)

0.8723076923076923

사이킷런 말고도 히스토그램 기반 그래디언트 부스팅 알고리즘을 구현한 라이브러리가 존재한다. 대표적으로 XGBoost와 LightGBM이 있다.

### XGBoost

In [26]:
xgb = XGBClassifier(tree_method='hist', random_state=42)

scores = cross_validate(xgb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


### LightGBM

In [27]:
lgb = LGBMClassifier(random_state=42)

scores = cross_validate(lgb, X_train, y_train,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
